# Using the [Yelp API](https://www.yelp.com/developers/documentation/v3/business_search)  to search a city by type.


## import libraries and api key
    * login['yelp-api-key']

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
KeyPath = %env CODINGDOJO
with open(KeyPath) as f: 
    login = json.load(f)
login.keys()

dict_keys(['MySQL', 'yelp-id', 'yelp-api-key'])

## Execute API call

In [4]:
#environment variables
location_state = 'CA'
location_city = 'Cerritos' 
term_call = 'artificial turf installation'
offset_call = 0
Data_Dir = f"Data/{location_state}/{location_city}/{term_call.replace(' ','_')}/"
json_file= 'data.json'
json_path = Data_Dir+json_file


In [5]:
# create data.json if does not exist in the appropriate directory
file_exists = os.path.isfile(json_path)
if file_exists == False:
    os.makedirs(Data_Dir,exist_ok=True)
    print(f"{json_file} not found. Saving empty list to file.")
    with open(json_path,'w') as f:
        json.dump([],f)  
else:
    print(f"{json_file} exists, will add to file")

data.json not found. Saving empty list to file.


In [6]:
# Open data.json to set offset based on previous results. 
with open(json_path,'r') as f:
    previous_results = json.load(f)
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [7]:
# connect with API key
yelp_api = YelpAPI(login['yelp-api-key'], timeout_s=5.0)

In [8]:
#get results from yelp API
results = yelp_api.search_query(location=location_city+', '+location_state,
                                term=term_call,
                                limit=50,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
# Calculate number of pages needed to get all results
# This number will be used for the progress bar 
try:
    n_pages = math.ceil(results['total']/ len(results['businesses']))
    print(n_pages)
except ZeroDivisionError:
    n_pages = 1
    print("No additional results")
    #No additional results.

19


In [14]:
first_run = True
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    if n_results == 0 or first_run == True:
    # append new results and save to file
        previous_results.extend(results['businesses'])
        with open(json_path,'w') as f:
            json.dump(previous_results,f)
        first_run = False
        n_results = len(previous_results)
    else:
        with open(json_path, 'r') as f:
            previous_results = json.load(f)
            n_results = len(previous_results)
            results = yelp_api.search_query(location=location_city+', '+location_state,
                                            term=term_call,limit=50,offset=n_results)
            previous_results.extend(results['businesses'])
            with open(json_path,'w') as f:
                json.dump(previous_results,f)
        

  0%|          | 0/19 [00:00<?, ?it/s]

In [15]:
# load final results
final_df = pd.read_json(json_path)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,q-E_xjVrRDPHN-DKcV00ZQ,grizzly-turf-orange-4,Grizzly Turf,https://s3-media3.fl.yelpcdn.com/bphoto/TMrO1c...,False,https://www.yelp.com/biz/grizzly-turf-orange-4...,360,"[{'alias': 'landscaping', 'title': 'Landscapin...",5.0,"{'latitude': 33.8186385, 'longitude': -117.828...",[],"{'address1': '', 'address2': None, 'address3':...",+18662378873,(866) 237-8873,22814.521408,NaN
1,6bBOsjaNvpON6EMjUEsVAQ,tough-turf-artesia-6,Tough Turf,https://s3-media2.fl.yelpcdn.com/bphoto/64FfNq...,False,https://www.yelp.com/biz/tough-turf-artesia-6?...,37,"[{'alias': 'artificialturf', 'title': 'Artific...",5.0,"{'latitude': 33.865012, 'longitude': -118.090254}",[],"{'address1': '18300 Gridley Rd', 'address2': N...",+17143428873,(714) 342-8873,1993.467410,NaN
2,1nSAoM71JOhjo8b4KxatKg,elite-artificial-grass-brea,Elite Artificial Grass,https://s3-media4.fl.yelpcdn.com/bphoto/XDO-8f...,False,https://www.yelp.com/biz/elite-artificial-gras...,201,"[{'alias': 'artificialturf', 'title': 'Artific...",5.0,"{'latitude': 33.92516580487358, 'longitude': -...",[],"{'address1': '411 W Lambert Rd', 'address2': '...",+17143320271,(714) 332-0271,16863.903785,NaN
3,2MU8A6VD3JWE4C0UevbpsA,mc-landscaping-lakewood-2,MC Landscaping,https://s3-media4.fl.yelpcdn.com/bphoto/BpINng...,False,https://www.yelp.com/biz/mc-landscaping-lakewo...,126,"[{'alias': 'landscaping', 'title': 'Landscapin...",4.5,"{'latitude': 33.85918742798152, 'longitude': -...",[],"{'address1': '5780 S St', 'address2': None, 'a...",+15622259460,(562) 225-9460,4713.074253,NaN
4,RtBCoWrbtjA6Kq6sQqFqUQ,always-green-turf-los-alamitos,Always Green Turf,https://s3-media4.fl.yelpcdn.com/bphoto/hY_eGI...,False,https://www.yelp.com/biz/always-green-turf-los...,55,"[{'alias': 'landscaping', 'title': 'Landscapin...",5.0,"{'latitude': 33.808837216795375, 'longitude': ...",[],"{'address1': '10611 Calle Lee D-129', 'address...",+17145877700,(714) 587-7700,6381.203603,NaN


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
913,IPxfXqanoeer_mWypyouNA,foregrounds-landscape-los-angeles-3,Foregrounds Landscape,https://s3-media4.fl.yelpcdn.com/bphoto/1i9RqU...,False,https://www.yelp.com/biz/foregrounds-landscape...,18,"[{'alias': 'landscaping', 'title': 'Landscapin...",5.0,"{'latitude': 34.02945, 'longitude': -118.35494}",[],"{'address1': '', 'address2': None, 'address3':...",+13106931703,(310) 693-1703,31614.761708,NaN
914,Ei0X-mUgfrT4-od3G5a9TA,be-green-landscaping-los-angeles-3,Be Green Landscaping,https://s3-media3.fl.yelpcdn.com/bphoto/RNWJvY...,False,https://www.yelp.com/biz/be-green-landscaping-...,5,"[{'alias': 'landscaping', 'title': 'Landscapin...",2.0,"{'latitude': 34.2499196, 'longitude': -118.600...",[],"{'address1': '21610 Lassen St', 'address2': 'S...",+18882478911,(888) 247-8911,65011.581178,NaN
915,q4wxINNNHL47AQTGYlNWIQ,global-landscaping-etiwanda,Global Landscaping,https://s3-media1.fl.yelpcdn.com/bphoto/Bl2kTH...,False,https://www.yelp.com/biz/global-landscaping-et...,3,"[{'alias': 'landscaping', 'title': 'Landscapin...",1.0,"{'latitude': 34.1425059921, 'longitude': -117....",[],"{'address1': None, 'address2': None, 'address3...",+19097583158,(909) 758-3158,59461.663416,NaN
916,lFMajkK_9t1W-r0B4P4MHA,reyes-waterfalls-el-cajon-4,Reyes Waterfalls,https://s3-media3.fl.yelpcdn.com/bphoto/WU6r9a...,False,https://www.yelp.com/biz/reyes-waterfalls-el-c...,12,"[{'alias': 'landscaping', 'title': 'Landscapin...",2.5,"{'latitude': 32.791524, 'longitude': -116.94795}",[],"{'address1': '333 S Anza St', 'address2': None...",+16196312808,(619) 631-2808,158442.493603,NaN
917,n_1XKmnp7rFbE8xJNjZtEw,avp-landscape-design-group-riverside-3,AVP Landscape Design Group,https://s3-media3.fl.yelpcdn.com/bphoto/OXlwXa...,False,https://www.yelp.com/biz/avp-landscape-design-...,5,"[{'alias': 'landscaping', 'title': 'Landscapin...",2.5,"{'latitude': 33.94259, 'longitude': -117.45739}",[],"{'address1': '', 'address2': None, 'address3':...",+16266186045,(626) 618-6045,57716.169411,NaN


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             918 non-null    object 
 1   alias          918 non-null    object 
 2   name           918 non-null    object 
 3   image_url      918 non-null    object 
 4   is_closed      918 non-null    bool   
 5   url            918 non-null    object 
 6   review_count   918 non-null    int64  
 7   categories     918 non-null    object 
 8   rating         918 non-null    float64
 9   coordinates    918 non-null    object 
 10  transactions   918 non-null    object 
 11  location       918 non-null    object 
 12  phone          918 non-null    object 
 13  display_phone  918 non-null    object 
 14  distance       918 non-null    float64
 15  price          11 non-null     object 
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 108.6+ KB


In [17]:
final_df.duplicated(subset='id').sum()

0

In [19]:
final_df.to_csv(f'{Data_Dir}data.csv',index=False)